In [ ]:
!git clone https://github.com/KU-DIC/LG_time_series_day04.git #코랩 사용

# [머신러닝 기반 시계열 분석 1 실습]
# Decision Tree & k-NN 
## [Decision Tree: Classification Tree - Example]

##### jupyter notebook 단축키

- ctrl+enter: 셀 실행   
- shift+enter: 셀 실행 및 다음 셀 이동   
- alt+enter: 셀 실행, 다음 셀 이동, 새로운 셀 생성
- a: 상단에 새로운 셀 만들기
- b: 하단에 새로운 셀 만들기
- dd: 셀 삭제(x: 셀 삭제)
- 함수 ( ) 안에서 shift+tab: arguments description. shift+tab+tab은 길게 볼 수 있도록

## 1. 모듈 불러오기

import 불러올 패키지명 as 그 패키지를 파이썬에서 사용할 이름

In [ ]:
""" 데이터 전처리 """
import numpy as np
import pandas as pd

""" 모델 생성, 학습, 평가 """
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

""" 시각화 """
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('/content/LG_time_series_day04/data/UniversalBank.csv')
# data = pd.read_csv('../Data/UniversalBank.csv') #로컬

data.head()

## 2. Universal bank 데이터
#### 설명변수 및 반응변수
- ID - 고객 고유번호 <br>
- Age - 고객의 연령 (연속형) <br>
- Experienc - 고객의 경력 (연속형) <br>
- Incom - 고객의 연간 수입 (연속형) <br>
- ZIP Code - 고객 주소 ZIP Code <br>
- Family - 고객의 가족 규모 (연속형) <br>
- CCAvg - 평균 신용 카드 지출 (연속형) <br>
- Education - 교육 수준 (범주형 - 1: 대학교 졸업, 2: 대학원 졸업, 3: 고급/전문) <br>
- Mortgag - 주택 모기지의 가치 (연속형) <br>
- SecuritiesAccount - 고객은 은행에 증권 계좌를 가지고 있는가? (범주형 - 0: 아니오, 1: 예) <br>
- CDAccount - 고객은 은행에 예금 계좌를 가지고 있는가?	(범주형 - 0: 아니오, 1: 예) <br>
- Online - 고객이 인터넷 뱅킹 시설을 사용하는가? (범주형 - 0: 아니오, 1: 예) <br>
- CreditCard - 고객이 Universal Bank에서 발행 한 신용 카드를 사용하는가? (범주형 - 0: 아니오, 1: 예)
- <span style="color:blue">PersonalLoan - 고객은 마지막 캠페인에 제공된 개인 대출을 수락 하였나? (범주형 - 0: 아니오, 1: 예) <b>[타겟 변수]</b> <br></span>

### ID와 ZIP Code는 예측에 필요하지 않은 변수이므로 제거

In [ ]:
data = data.drop(['ID','ZIP Code'], axis=1)

### Education 변수에 대한 더미 변수 생성
- Education - 교육 수준 (범주형 - 1: 대학교 졸업, 2: 대학원 졸업, 3: 고급/전문) 

In [ ]:
# Education: dummies
data = pd.get_dummies(data, columns=['Education'], drop_first=True)
data.head()

### X, y로 구분

In [ ]:
X = data.drop('Personal Loan', axis=1)
y = data['Personal Loan']

### 클래스의 분포를 확인

In [ ]:
plt.figure(figsize=(4, 5))
sns.countplot(y)
plt.show()

### 학습-평가 데이터셋 구성
- Train:Validation:Test = 60:20:20
- 클래스 비율 유지: train_test_split 함수 내 stratify 옵션
- randomstate = 2023

In [ ]:
random_state = 2023
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=2/10, random_state=random_state, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=2/8,
                                                      random_state=random_state, stratify=y_train)

In [ ]:
print('클래스별 데이터 개수: Train')
print(y_train.value_counts())

print('\n ----------------------- \n')
print('클래스별 데이터 개수: Validation')
print(y_valid.value_counts())

print('\n ----------------------- \n')
print('클래스별 데이터 개수: Test')
print(y_test.value_counts())

### Decision Tree Classifier
- 최적의 max_depth를 선택해야 함
- Validation을 사용하여 accuracy, F1-Score를 고려하여 선정

In [ ]:
max_depths = list(range(1, 10)) + [None]
print(max_depths)

In [ ]:
# 평가 지표 저장
acc_valid = []
f1_valid = []

In [ ]:
for max_depth in max_depths:
                      
    # 모델 학습
    model = DecisionTreeClassifier(max_depth=max_depth)
    model.fit(X_train, y_train)
    
    # validation 예측
    y_valid_pred = model.predict(X_valid)
    
    # 모델 평가 결과 저장
    acc = accuracy_score(y_valid, y_valid_pred)
    f1 = f1_score(y_valid, y_valid_pred)
    
    acc_valid.append(acc)
    f1_valid.append(f1)

시각화를 위해 max_depths를 문자열로 바꿔준다

In [ ]:
# Decision Tree depth list
xticks = list(map(str, max_depths))
print(xticks)

In [ ]:
# Decision Tree depth에 따른 accuracy
fig, ax = plt.subplots(figsize=(15, 6))
#fig.subplots_adjust(right=0.75)

ax.plot(range(len(max_depths)), acc_valid, color='red', marker='o')
ax.set_ylabel('accuracy', color='red', fontsize=12)

ax2 = ax.twinx()
ax2.plot(range(len(max_depths)), f1_valid, color='blue', marker='s')
ax2.set_ylabel('f1', color='blue', fontsize=12)

plt.xticks(range(len(max_depths)), xticks)
plt.show()

### <span style="color:blue">max_depth = 4로 설정</b> <br></span>

In [ ]:
model = DecisionTreeClassifier(max_depth=4)
model.fit(X_train, y_train)

In [ ]:
# 에측 결과 산출
y_test_pred = model.predict(X_test)

# Confusion Matrix
cm = confusion_matrix(y_test, y_test_pred)
cm = pd.DataFrame(cm)

# Accuracy, F1-Score
acc = accuracy_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

print('- Accuracy (Test) : {:.3}'.format(acc))
print('- F1 score (Test) : {:.3}'.format(f1))

In [ ]:
# 시각화
plt.figure(figsize=(6, 5))
sns.heatmap(data=cm, annot=True, annot_kws={'size': 15}, fmt='d', cmap='Blues')
plt.title('Acc = {:.3f} & F1 = {:.3f}'.format(acc, f1))
plt.show()

## 규칙 시각화

In [ ]:
plt.figure(figsize=(20, 10))
plot_tree(decision_tree=model, filled=True)
plt.show()

## 변수 중요도: 공간을 split하는데 기여한 정도를 정량적으로 표현

In [ ]:
# 변수 중요도
importances = model.feature_importances_

# 내림차순으로 정렬하기 위한 index
index = np.argsort(importances)[::-1]

In [ ]:
plt.figure(figsize=(8, 6))
plt.title('Feature Importances')
plt.bar(range(X.shape[1]),
        importances[index],
        align='center')
plt.xticks(range(X.shape[1]), X.columns, rotation=90)
plt.xlim([-1, X.shape[1]])
plt.show()